In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [3]:
import sklearn.feature_extraction as sklearn_text

## Tokenizing and term document matrix creation 

In [4]:
??URLs

In [5]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/Users/ricardofernandez/.fastai/data/imdb_sample')

In [6]:
path.ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb_sample/texts.csv')]

In [7]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [8]:
from collections import Counter

In [9]:
Counter(df.is_valid)

Counter({False: 800, True: 200})

In [10]:
Counter(df.label)

Counter({'negative': 524, 'positive': 476})

In [11]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                 .split_from_df(col=2)
                 .label_from_df(cols=0))

In [12]:
movie_reviews.train.x[0]

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

In [13]:
movie_reviews.train.y[0]

Category negative

In [14]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [15]:
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

(6008, 19161)

In [16]:
movie_reviews.vocab.stoi['language']

917

In [17]:
movie_reviews.vocab.itos[917]

'language'

In [18]:
movie_reviews.vocab.itos[20:30]

['that', 'this', '"', "'s", '\n \n ', '-', 'was', 'as', 'for', 'movie']

In [19]:
movie_reviews.vocab.itos[6007]

'sollett'

In [20]:
movie_reviews.vocab.itos[:20]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

In [21]:
movie_reviews.train.x[5].data.shape

(186,)

In [22]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi["Rrrick"]]

'xxunk'

In [23]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi["language"]]

'language'

In [24]:
t = movie_reviews.train[0][0]

In [25]:
type(t)

fastai.text.data.Text

In [26]:
t.data[:30]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776,    5, 1800,   95,   37,   85,  191,   64,  935,
          0, 2738,  517,   18,   21,   11,   84, 2417,  192,   88, 3777,   64])

## Creating our term-document matrix 

In [27]:
c = Counter([4,3,8,8,4,8])

In [28]:
c

Counter({4: 2, 3: 1, 8: 3})

In [29]:
c.values()

dict_values([2, 1, 3])

In [30]:
c.keys()

dict_keys([4, 3, 8])

## Our version of CountVectorizer 

In [31]:
Counter((movie_reviews.valid.x)[0].data)

Counter({2: 1,
         5: 32,
         21: 3,
         71: 1,
         189: 1,
         748: 1,
         288: 1,
         285: 1,
         63: 2,
         221: 1,
         666: 2,
         59: 1,
         13: 4,
         2705: 1,
         14: 6,
         2875: 1,
         11: 10,
         18: 2,
         358: 1,
         0: 32,
         77: 1,
         15: 6,
         478: 1,
         1833: 1,
         50: 3,
         9: 10,
         319: 1,
         6: 1,
         2743: 1,
         12: 1,
         115: 1,
         4126: 1,
         197: 2,
         1331: 1,
         25: 2,
         324: 1,
         10: 7,
         3963: 1,
         16: 4,
         74: 1,
         24: 3,
         2817: 1,
         5821: 1,
         2595: 1,
         710: 1,
         3429: 1,
         84: 1,
         149: 1,
         20: 1,
         26: 1,
         605: 1,
         378: 1,
         1057: 1,
         251: 1,
         258: 1,
         1346: 1,
         194: 1,
         239: 1,
         49: 1,
         27

In [32]:
movie_reviews.vocab.itos[5]

'xxmaj'

In [33]:
movie_reviews.vocab.itos[6]

'xxup'

In [34]:
(movie_reviews.valid.x)[0]

Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
 
  xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
 
  xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk , xxma

In [35]:
a = [4,3]
a.extend([3,4])

In [36]:
a

[4, 3, 3, 4]

In [37]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)
    
    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
      
    #print(len(indptr))
    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                  shape=(len(indptr)-1, vocab_len),
                                  dtype=int)

In [38]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x,
                                  len(movie_reviews.vocab.itos))

CPU times: user 43.3 ms, sys: 4.06 ms, total: 47.3 ms
Wall time: 45.4 ms


In [39]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x,
                                  len(movie_reviews.vocab.itos))

CPU times: user 132 ms, sys: 8.97 ms, total: 141 ms
Wall time: 137 ms


In [40]:
trn_term_doc.shape

(800, 6008)

In [41]:
 trn_term_doc[:,-10:]

<800x10 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [42]:
val_term_doc.shape

(200, 6008)

### More data exploration 

In [43]:
movie_reviews.vocab.itos[-1:]

['sollett']

In [44]:
val_term_doc.todense()[:10,:10]

matrix([[32,  0,  1,  0, ...,  1,  0,  0, 10],
        [ 9,  0,  1,  0, ...,  1,  0,  0,  7],
        [ 6,  0,  1,  0, ...,  0,  0,  0, 12],
        [78,  0,  1,  0, ...,  0,  0,  0, 44],
        ...,
        [ 8,  0,  1,  0, ...,  0,  0,  0,  8],
        [43,  0,  1,  0, ...,  8,  1,  0, 25],
        [ 7,  0,  1,  0, ...,  1,  0,  0,  9],
        [19,  0,  1,  0, ...,  2,  0,  0,  5]])

In [45]:
c = Counter(movie_reviews.valid.x[3].data)
for i in range(10):
    print(f"index {i} value {c[i]}")

index 0 value 78
index 1 value 0
index 2 value 1
index 3 value 0
index 4 value 0
index 5 value 52
index 6 value 0
index 7 value 0
index 8 value 0
index 9 value 44


In [46]:
movie_reviews.vocab.itos[3]

'xxeos'

In [47]:
review = movie_reviews.valid.x[1]; review

Text xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk

In [48]:
val_term_doc[1,movie_reviews.vocab.stoi["late"]]

2

In [49]:
val_term_doc

<200x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 27848 stored elements in Compressed Sparse Row format>

In [50]:
val_term_doc[1]

<1x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 81 stored elements in Compressed Sparse Row format>

In [51]:
val_term_doc[1].sum(), movie_reviews.valid[1][0].data.shape

(144, (144,))

In [52]:
string = ""
for i in review.data:
    string += " " + movie_reviews.vocab.itos[i]
print(string[1:])

xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk


In [53]:
np.where(val_term_doc[1].todense() != 0)[1].shape

(81,)

In [54]:
len(set(review.data))

81

In [55]:
movie_reviews.vocab.itos[1000:1020]

['state',
 'street',
 'impossible',
 'clever',
 'development',
 'concept',
 'william',
 'worked',
 'adventure',
 'church',
 'unlike',
 'hold',
 'lots',
 'premise',
 'shooting',
 'washington',
 'sick',
 'effect',
 'waiting',
 'singing']

In [56]:
len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)

13154

In [57]:
movie_reviews.vocab.itos[0]

'xxunk'

In [58]:
unk = []
for word, num in movie_reviews.vocab.stoi.items():
    if num == 0:
        unk.append(word)

In [59]:
len(unk)

13155

In [60]:
unk[:30]

['xxunk',
 'bleeping',
 'pert',
 'ticky',
 'schtick',
 'whoosh',
 'banzai',
 'chill',
 'wooofff',
 'cheery',
 'superstars',
 'fashionable',
 'cruelly',
 'separating',
 'mistreat',
 'tensions',
 'religions',
 'baseness',
 'nobility',
 'puro',
 'disowned',
 'option',
 'faults',
 'dignified',
 'realisation',
 'reconciliation',
 'mrs',
 'iyer',
 'heartbreaking',
 'histories']

## Naive Bayes 

In [61]:
movie_reviews.y.classes

['negative', 'positive']

In [62]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [66]:
val_y

CategoryList (200 items)
positive,positive,positive,positive,positive
Path: /Users/ricardofernandez/.fastai/data/imdb_sample

In [68]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [71]:
positive, negative

(1, 0)

In [73]:
v = movie_reviews.vocab
v.itos[0]

'xxunk'

grab all rows with negative sentiment and sum total word counts for each word

In [75]:
x[y.items == negative]

<417x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 60554 stored elements in Compressed Sparse Row format>

In [77]:
np.asarray(x[y.items == negative].sum(0))

array([[7154,    0,  417,    0, ...,    0,    3,    3,    3]], dtype=int64)

In [78]:
np.squeeze(np.asarray(x[y.items == negative].sum(0)))

array([7154,    0,  417,    0, ...,    0,    3,    3,    3], dtype=int64)

In [79]:
np.squeeze(np.asarray(x[y.items == positive].sum(0)))

array([6471,    0,  383,    0, ...,    3,    0,    0,    0], dtype=int64)

In [80]:
p1 = np.squeeze(np.asarray(x[y.items == positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items == negative].sum(0)))

In [81]:
p1[:10]

array([ 6471,     0,   383,     0,     0, 10267,   674,    57,     0,  5260], dtype=int64)

In [82]:
len(p1), len(p0), len(v.itos)

(6008, 6008, 6008)

In [86]:
p1[v.stoi["loved"]]/p0[v.stoi["loved"]]

2.4166666666666665

In [87]:
p1[v.stoi["hate"]]/p0[v.stoi["hate"]]

0.38461538461538464

In [89]:
v.stoi['hated']

1977

In [90]:
a = np.argwhere((x[:,1977] > 0))

In [91]:
a

array([[ 15,   0],
       [ 49,   0],
       [304,   0],
       [351,   0],
       [393,   0],
       [612,   0],
       [695,   0],
       [773,   0]], dtype=int32)

In [92]:
a.shape

(8, 2)

In [93]:
movie_reviews.train.x[15]

Text xxbos xxmaj as if the storyline was n't depressing enough , this movie shows xxunk being xxunk graphically in a slaughterhouse for all of five minutes while the protagonist is xxunk her early life as a butcher . xxmaj weird stuff . xxmaj then there 's the core premise of the hero / heroine who goes and cuts his dick off because a he 's xxunk - ten with at work says he would have gone with him if he was a girl . xxmaj is this person a psycho , a xxunk , just a doomed queen who takes things too far ? xxmaj and what sort of xxunk childhood did he have ? xxmaj just that he did n't get adopted and had to live it out with xxunk who at first loved him and then later hated him because he was xxunk . xxmaj he tries to explain to us the reasons he did what he did , but it 's really really so hard to xxunk . xxmaj such sad and unusual self destruction . xxmaj was it supposed to be funny ? xxmaj what was it all about really ?

In [94]:
a = np.argwhere((x[:,1977] > 0))[:,0]; a

array([ 15,  49, 304, 351, 393, 612, 695, 773], dtype=int32)

In [95]:
b = np.argwhere(y.items==positive)[:,0]; b

array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

In [96]:
len(a), len(b)

(8, 383)

In [97]:
set(a).intersection(set(b))

{393, 612, 695}

In [98]:
review = movie_reviews.train.x[695]
review.text

"xxbos xxmaj xxunk , yeah this episode is extremely underrated . \n \n  xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n \n  i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n \n  xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n \n  i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."

In [99]:
v.stoi['loved']

535

In [104]:
a = np.argwhere((x[:,535] > 0))[:,0]; a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785], dtype=int32)

In [105]:
b = np.argwhere(y.items==negative)[:,0]; b

array([  0,   2,   4,   5, ..., 795, 796, 798, 799])

In [106]:
set(a).intersection(set(b))

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [107]:
review = movie_reviews.train.x[792]
review.text

'xxbos xxmaj this is not really a zombie film , if we \'re xxunk zombies as the dead walking around . xxmaj here the protagonist , xxmaj xxunk xxmaj louque ( played by an xxunk young xxmaj dean xxmaj xxunk ) , xxunk control of a method to create zombies , though in fact , his \' method \' is to mentally project his thoughts and control other living people \'s minds turning them into xxunk slaves . xxmaj this is an interesting concept for a movie , and was done much more effectively by xxmaj xxunk xxmaj lang in his series of \' xxmaj dr. xxmaj mabuse \' films , including \' xxmaj dr. xxmaj mabuse the xxmaj xxunk \' ( xxunk ) and \' xxmaj the xxmaj testament of xxmaj dr. xxmaj mabuse \' ( 1933 ) . xxmaj here it is unfortunately xxunk to his quest to regain the love of his former fiancée , xxmaj claire xxmaj duvall ( played by the xxmaj anne xxmaj xxunk look alike with a bad xxunk , xxmaj dorothy xxmaj stone ) which is really the major theme . \n \n  xxmaj the movie has an intriguing begi

## Applying Naive Bayes 

In [117]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

below is not correct for bayes as we should be calculating p(w|positive) = p(w,positive)/sum(p(w_k, positive)) over vocab.

In [121]:
alpha = 1
pr1 = (p1+alpha)/((y.items==positive).sum() + alpha)
pr0 = (p0+alpha)/((y.items==negative).sum() + alpha)

In [122]:
pr1[pr1<0]

array([], dtype=float64)

In [123]:
r = np.log(pr1/pr0); r

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

In [124]:
biggest = np.argpartition(r, -10)[-10:]
smallest = np.argpartition(r, 10)[:10]

In [127]:
# most positive words
[v.itos[k] for k in biggest]

['sport',
 'davies',
 'gilliam',
 'fanfan',
 'biko',
 'felix',
 'noir',
 'jabba',
 'astaire',
 'jimmy']

In [128]:
np.argmax(trn_term_doc[:,v.stoi['biko']])

515

In [129]:
movie_reviews.train.x[515]

Text xxbos " xxmaj the xxmaj true xxmaj story xxmaj of xxmaj the xxmaj friendship xxmaj that xxmaj shook xxmaj south xxmaj africa xxmaj and xxmaj xxunk xxmaj the xxmaj world . " 
 
  xxmaj richard xxmaj attenborough , who directed " a xxmaj bridge xxmaj too xxmaj far " and " xxmaj gandhi " , wanted to bring the story of xxmaj steve xxmaj biko to life , and the journey and trouble that xxunk xxmaj donald xxmaj woods went through in order to get his story told . xxmaj the films uses xxmaj wood 's two books for it 's information and basis - " xxmaj biko " and " xxmaj asking for xxmaj trouble " . 
 
  xxmaj the film takes place in the late 1970 's , in xxmaj south xxmaj africa . xxmaj south xxmaj africa is in the grip of the terrible apartheid , which keeps the blacks separated from the whites and xxunk the whites as the superior race . xxmaj the blacks are forced to live in xxunk on the xxunk of the cities and xxunk , and they come under frequent xxunk by the police and the army . xxmaj w

above is a long review with the word beko used many times, you can see that makes beko a strong feature but this is not correct. This is a flaw that has to be corrected.

negative reveiws

In [130]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [131]:
np.argmax(trn_term_doc[:,v.stoi["soderbergh"]])

434

In [132]:
movie_reviews.train.x[434]

Text xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of " xxmaj at xxmaj the xxmaj movies " in taking xxmaj steven xxmaj soderbergh to task . 
 
  xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after xxunk years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside " edgy " projects . 
 
  xxmaj none of this excuses him this present , almost diabolical failure . xxmaj as xxmaj david xxmaj xxunk xxunk , " two parts of xxmaj che do n't ( even ) make a whole " . 
 
  xxmaj epic xxunk in name only , xxmaj che(2008 ) barely qualifies as a feature film 

## Continuing with Naive Bayes 

In [133]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [134]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

In [135]:
val_term_doc.shape, r.shape

((200, 6008), (6008,))

In [136]:
preds = (val_term_doc @ r  + b) > 0

In [137]:
(preds == val_y.items).mean()

0.645

In [150]:
np.random.randint(low=0,high=2,size=len(val_y))

array([0, 1, 0, 1, ..., 1, 1, 1, 0])

In [151]:
(np.random.randint(low=0,high=2,size=len(val_y)) == val_y.items).mean()

0.435

its a start but not that great

## Switching to full data set

In [152]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_clas'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/unsup'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/README'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_lm'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train')]

In [153]:
(path/'train').ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/neg'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/pos'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/labeledBow.feat')]

In [154]:
reviews_full = (TextList.from_folder(path)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos']))
             #label them all with their folders

In [155]:
len(reviews_full.train), len(reviews_full.valid)

(25000, 25000)

In [156]:
v = reviews_full.vocab

In [157]:
v.itos[100:110]

['bad',
 'people',
 'will',
 'other',
 'also',
 'into',
 'first',
 'because',
 'great',
 'how']

In [158]:
%%time
val_term_doc = get_term_doc_matrix(reviews_full.valid.x, len(reviews_full.vocab.itos))

CPU times: user 3.82 s, sys: 155 ms, total: 3.97 s
Wall time: 3.9 s


In [163]:
%%time
trn_term_doc = get_term_doc_matrix(reviews_full.train.x, len(reviews_full.vocab.itos))

CPU times: user 3.89 s, sys: 184 ms, total: 4.07 s
Wall time: 4.03 s


In [164]:
trn_term_doc.shape, val_term_doc.shape

((25000, 38456), (25000, 38456))

save the data

In [165]:
scipy.sparse.save_npz("trn_term_doc.npz", trn_term_doc)

In [166]:
scipy.sparse.save_npz("val_term_doc.npz", val_term_doc)

In [167]:
trn_term_doc = scipy.sparse.load_npz("trn_term_doc.npz")
val_term_doc = scipy.sparse.load_npz("val_term_doc.npz")

## Naive Bayes on full dataset

In [168]:
x=trn_term_doc
y=reviews_full.train.y

val_y = reviews_full.valid.y.items

In [169]:
x

<25000x38456 sparse matrix of type '<class 'numpy.int64'>'
	with 3716265 stored elements in Compressed Sparse Row format>

In [170]:
positive = y.c2i['pos']
negative = y.c2i['neg']

In [171]:
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [172]:
p1[:20]

array([ 28450,      0,  12500,      0,      0, 342619,  20464,   1338,      7, 173122, 138000, 143763,  89570,  83404,
        76828,  66715,  58510,  47896,  50177,  40451], dtype=int64)

In [173]:
def neg_pos_given_word(word):
    print(p0[v.stoi[word]]/p1[v.stoi[word]])

In [174]:
neg_pos_given_word('hated')

2.051546391752577


In [175]:
neg_pos_given_word('liked')

0.6424702058504875


In [176]:
neg_pos_given_word('loved')

0.3139963167587477


In [177]:
neg_pos_given_word('best')

0.48538961038961037


In [178]:
neg_pos_given_word('worst')

9.837301587301587


In [179]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [180]:
r = np.log(pr1/pr0)

In [181]:
r[v.stoi['hated']]

-0.7133498878774648

In [182]:
r[v.stoi['loved']]

1.1563661500586044

In [183]:
r[v.stoi['worst']]

-2.2826243504315076

In [184]:
r[v.stoi['best']]

0.7225576052173609

## Back to Naive Bayes

In [196]:
negative = y.c2i['neg']
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [197]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)
r = np.log(pr1/pr0)

In [198]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()); b

0.0

In [199]:
preds = (val_term_doc @ r + b) > 0

In [200]:
(preds == val_y).mean()

0.8084

In [201]:
x[y.items==negative].sum(), x.shape

(3717801, (25000, 38456))

In [202]:
pr1 = (p1+1) / (x[y.items==positive].sum() + x.shape[1])
pr0 = (p0+1) / (x[y.items==negative].sum() + x.shape[1])
r = np.log(pr1/pr0)

In [203]:
preds = (val_term_doc @ r + b) > 0
(preds == val_y).mean()

0.80376

interesting the correct way was not much different

## Binarized Naive Bayes 

maybe it only matters whether a word is in the review or not

In [204]:
x=trn_term_doc.sign()
y=reviews_full.train.y

In [205]:
x.todense()[:10,:10]

matrix([[0, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 1, 0, 1],
        ...,
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1]])

In [206]:
negative = y.c2i['neg']
positive = y.c2i['pos']

In [207]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [208]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [209]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

preds = (val_term_doc.sign() @ r + b) > 0

In [210]:
(preds == val_y).mean()

0.82924

## Logistic regression 

In [211]:
from sklearn.linear_model import LogisticRegression

In [219]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_term_doc, y.items.astype(int))
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.88248

In [220]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_term_doc.sign(), y.items.astype(int))
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.88536

## Trigram with NB features 

In [221]:
v.stoi["xxmaj"]

5

In [222]:
v.itos[5]

'xxmaj'

In [223]:
p_ab = 99.9/100
p_b = 0.01/100
p_abc = 0.1/100

In [226]:
p_a = p_ab*p_b + p_abc*(1-p_b)
p_ba = p_b * p_ab/p_a
p_ba*100

9.083469721767596

In [234]:
1/5,2/5,4/5,8/5-1,2*(8/5-1)-1

(0.2, 0.4, 0.8, 0.6000000000000001, 0.20000000000000018)

In [235]:
def f(x):
    if x<=1/2:
        return 2*x
    if x>1/2:
        return 2*x-1

In [239]:
x = 1/10
for i in range(5):
    x = f(x)
    print(x)

0.2
0.4
0.8
0.6000000000000001
0.20000000000000018


In [240]:
2/5

0.4

### ngrams

In [241]:
path = untar_data(URLs.IMDB_SAMPLE)

In [242]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0))

In [243]:
v = movie_reviews.vocab.itos

In [244]:
vocab_len = len(v)

### create train matrix 

In [247]:
movie_reviews.train.x[0].data

array([   2,    5, 4619,   25, ...,   10,    5,    0,   52])

In [251]:
Counter(movie_reviews.train.x[0].data).keys()

dict_keys([2, 5, 4619, 25, 0, 867, 52, 3776, 1800, 95, 37, 85, 191, 64, 935, 2738, 517, 18, 21, 11, 84, 2417, 192, 88, 3777, 1801, 127, 10, 269, 15, 273, 73, 26, 9, 1360, 35, 1213, 1144, 1145, 2418, 91, 63, 245, 14, 1361, 1447, 65, 40, 796, 103, 72, 99, 534, 616, 48, 282, 54, 90, 219, 228, 43, 13, 3778, 3779, 355, 492])

In [264]:
min_n = 1
max_n = 3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

In [265]:
ngramtoi

{}

In [267]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) not in ngramtoi:
                if len(ngram)==1:
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [269]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

### Looking at our data 

In [270]:
len(ngramtoi), len(itongram)

(260374, 260374)

In [272]:
itongram[20005]

array([ 15,   9, 710])

In [273]:
ngramtoi[str(np.array([15,9,710]))]

20005

In [274]:
itongram[100000]

array([1140,   33])

In [275]:
v[1140], v[33]

('hate', 'you')

### create valid matrix 

In [276]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [277]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [278]:
valid_ngram_doc_matrix

<200x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 121600 stored elements in Compressed Sparse Row format>

In [279]:
train_ngram_doc_matrix

<800x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 678951 stored elements in Compressed Sparse Row format>

### Save data

In [280]:
scipy.sparse.save_npz("train_ngram_matrix.npz", train_ngram_doc_matrix)

In [281]:
scipy.sparse.save_npz("valid_ngram_matrix.npz", valid_ngram_doc_matrix)

In [282]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ngramtoi.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load data

In [283]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_matrix.npz")

In [284]:
with open('itongram.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
with open('ngramtoi.pickle', 'rb') as handle:
    b = pickle.load(handle)

### Naive Bayes

In [285]:
x=train_ngram_doc_matrix
y=movie_reviews.train.y

In [286]:
positive = y.c2i["positive"]
negative = y.c2i['negative']

In [287]:
x

<800x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 678951 stored elements in Compressed Sparse Row format>

In [290]:
k = x.shape[1]; k

260374

In [291]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [301]:
xx = x[:k]

In [302]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [303]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [304]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [305]:
r = np.log(pr1/pr0)

In [306]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

In [307]:
b

-0.08505123261815539

In [308]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [309]:
r.shape

(260374,)

In [310]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

In [311]:
preds = pre_preds.T > 0

In [312]:
preds.shape

(200,)

In [313]:
preds[:10]

array([ True,  True,  True,  True, False,  True,  True, False,  True, False])

In [315]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [316]:
(preds == valid_labels).mean()

0.76

better result then only using unigrams

## Binarized Naive Bayes

In [317]:
trn_x_ngram_sgn = train_ngram_doc_matrix.sign()
val_x_ngram_sgn = valid_ngram_doc_matrix.sign()

In [318]:
xx = trn_x_ngram_sgn[:k]

In [319]:
trn_x_ngram_sgn.shape, xx.shape

((800, 260374), (800, 260374))

In [320]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [321]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [322]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

pre_preds = val_x_ngram_sgn @ r.T + b
preds = pre_preds.T>0

In [323]:
(preds==valid_labels).mean()

0.735

## Logisitic Regression 

In [324]:
from sklearn.linear_model import LogisticRegression

### use CountVectorizer to compare 

In [325]:
from sklearn.feature_extraction.text import CountVectorizer

In [327]:
??noop

In [328]:
veczr = CountVectorizer(ngram_range=(1,3), preprocessor=noop,
                        tokenizer=noop, max_features=800000)

In [329]:
docs = movie_reviews.train.x

In [330]:
train_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.train.x]

In [331]:
train_words[0]

['xxbos',
 'xxmaj',
 'un',
 '-',
 'xxunk',
 '-',
 'believable',
 '!',
 'xxmaj',
 'meg',
 'xxmaj',
 'ryan',
 'does',
 "n't",
 'even',
 'look',
 'her',
 'usual',
 'xxunk',
 'lovable',
 'self',
 'in',
 'this',
 ',',
 'which',
 'normally',
 'makes',
 'me',
 'forgive',
 'her',
 'shallow',
 'xxunk',
 'acting',
 'xxunk',
 '.',
 'xxmaj',
 'hard',
 'to',
 'believe',
 'she',
 'was',
 'the',
 'producer',
 'on',
 'this',
 'dog',
 '.',
 'xxmaj',
 'plus',
 'xxmaj',
 'kevin',
 'xxmaj',
 'kline',
 ':',
 'what',
 'kind',
 'of',
 'suicide',
 'trip',
 'has',
 'his',
 'career',
 'been',
 'on',
 '?',
 'xxmaj',
 'xxunk',
 '...',
 'xxmaj',
 'xxunk',
 '!',
 '!',
 '!',
 'xxmaj',
 'finally',
 'this',
 'was',
 'directed',
 'by',
 'the',
 'guy',
 'who',
 'did',
 'xxmaj',
 'big',
 'xxmaj',
 'xxunk',
 '?',
 'xxmaj',
 'must',
 'be',
 'a',
 'replay',
 'of',
 'xxmaj',
 'jonestown',
 '-',
 'hollywood',
 'style',
 '.',
 'xxmaj',
 'xxunk',
 '!']

In [332]:
valid_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.valid.x]

In [333]:
for i in range(5):
    print(len(train_words[i]))

103
463
220
185
397


In [334]:
%%time
train_ngram_doc = veczr.fit_transform(train_words)

CPU times: user 1.44 s, sys: 69.5 ms, total: 1.51 s
Wall time: 1.54 s


In [335]:
train_ngram_doc

<800x260373 sparse matrix of type '<class 'numpy.int64'>'
	with 565680 stored elements in Compressed Sparse Row format>

In [336]:
veczr.vocabulary_

{'xxbos': 235215,
 'xxmaj': 235589,
 'un': 217511,
 '-': 14660,
 'xxunk': 247952,
 'believable': 50420,
 '!': 593,
 'meg': 134438,
 'ryan': 171955,
 'does': 72621,
 "n't": 141191,
 'even': 78281,
 'look': 129009,
 'her': 101674,
 'usual': 219401,
 'lovable': 129862,
 'self': 175864,
 'in': 110056,
 'this': 206619,
 ',': 8800,
 'which': 228198,
 'normally': 145182,
 'makes': 131546,
 'me': 133646,
 'forgive': 88662,
 'shallow': 177158,
 'acting': 27682,
 '.': 16836,
 'hard': 97855,
 'to': 210364,
 'believe': 50457,
 'she': 177306,
 'was': 222309,
 'the': 193798,
 'producer': 164483,
 'on': 152319,
 'dog': 72926,
 'plus': 162127,
 'kevin': 122604,
 'kline': 123461,
 ':': 20359,
 'what': 226855,
 'kind': 123231,
 'of': 147512,
 'suicide': 188353,
 'trip': 215808,
 'has': 98123,
 'his': 103526,
 'career': 58691,
 'been': 49234,
 '?': 20987,
 '...': 18342,
 'finally': 85368,
 'directed': 71212,
 'by': 56556,
 'guy': 96410,
 'who': 229023,
 'did': 70468,
 'big': 51595,
 'must': 140407,
 'be'

In [337]:
val_ngram_doc = veczr.transform(valid_words)

In [338]:
val_ngram_doc

<200x260373 sparse matrix of type '<class 'numpy.int64'>'
	with 93552 stored elements in Compressed Sparse Row format>

In [339]:
vocab = veczr.get_feature_names()

In [340]:
vocab[200000:200005]

['the same cult',
 'the same dance',
 'the same date',
 'the same day',
 'the same diner']

### Binarized Naive Bayes, using ngrams form CountVectorizer 

In [341]:
y = movie_reviews.train.y

In [343]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(train_ngram_doc.sign(), y.items);

preds = m.predict(val_ngram_doc.sign())
(preds.T==valid_labels).mean()

0.83

In [344]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(train_ngram_doc, y.items);

preds = m.predict(val_ngram_doc)
(preds.T==valid_labels).mean()

/Users/ricardofernandez/anaconda3/envs/deeplearning/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.78

In [345]:
m2 = LogisticRegression(C=0.1, dual=True)
m2.fit(trn_x_ngram_sgn, y.items)

LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [346]:
preds = m2.predict(val_x_ngram_sgn)
(preds.T==valid_labels).mean()

0.83

In [347]:
m2 = LogisticRegression(C=0.0001, dual=True, max_iter=50000)
m2.fit(train_ngram_doc_matrix, y.items)

LogisticRegression(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [348]:
preds = m2.predict(valid_ngram_doc_matrix)
(preds.T==valid_labels).mean()

0.675